<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/HOPuanlama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener matplotlib openpyxl TvDatafeed

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tradingview_screener import get_all_symbols
import warnings

# Uyarıları gizle
warnings.simplefilter(action='ignore', category=FutureWarning)

# Kullanıcının tarama yapacağı zaman dilimini seçmesini sağlayan fonksiyon
def get_interval_choice(count=2):
    intervals = {
        '1': ('15 Dakika', Interval.in_15_minute),
        '2': ('30 Dakika', Interval.in_30_minute),
        '3': ('45 Dakika', Interval.in_45_minute),
        '4': ('1 Saat', Interval.in_1_hour),
        '5': ('2 Saat', Interval.in_2_hour),
        '6': ('4 Saat', Interval.in_4_hour),
        '7': ('Günlük', Interval.in_daily),
        '8': ('Haftalık', Interval.in_weekly),
        '9': ('Aylık', Interval.in_monthly)
    }

    print("Lütfen zaman dilimi(lerini) seçin:")
    for key, value in intervals.items():
        print(f"{key}: {value[0]}")

    # Kullanıcının birinci zaman dilimini seçmesi
    choice_1 = input("Birinci zaman dilimini seçin (1-9): ")
    interval_1 = intervals.get(choice_1, ('Günlük', Interval.in_daily))

    if count == 2:
        # Kullanıcının ikinci zaman dilimini seçmesi
        choice_2 = input("İkinci zaman dilimini seçin (1-9): ")
        interval_2 = intervals.get(choice_2, ('Günlük', Interval.in_daily))
        return [interval_1, interval_2]

    return [interval_1]

# Kullanıcıdan zaman dilimi sayısını seçmesini sağlayan fonksiyon
def get_interval_count_choice():
    print("Kaç farklı zaman diliminde tarama yapmak istiyorsunuz?")
    print("1: Tek Zaman Dilimi")
    print("2: İki Zaman Dilimi")

    choice = input("Seçiminizi yapın (1-2): ")

    if choice == '1':
        return 1
    elif choice == '2':
        return 2
    else:
        print("Geçersiz seçim, varsayılan olarak iki zaman dilimi seçildi.")
        return 2

# Kullanıcıdan hangi tür hareketli ortalamaları seçeceğini soran fonksiyon
def get_moving_average_choice():
    print("Hangi hareketli ortalamaları kullanmak istersiniz?")
    print("1: SMA")
    print("2: EMA")
    print("3: WMA")
    print("4: Hull")

    choices = input("Seçimlerinizi yapın (örneğin: 1,2): ")
    selected_choices = choices.split(',')

    return [int(choice) for choice in selected_choices if choice in ['1', '2', '3', '4']]

# Hareketli ortalamaları hesaplayan fonksiyon
def calculate_moving_averages(data, ma_types):
    moving_averages = {}

    for ma_type in ma_types:
        if ma_type == 1:  # SMA
            moving_averages['8 SMA'] = data['Close'].rolling(window=8).mean()
            moving_averages['20 SMA'] = data['Close'].rolling(window=20).mean()
            moving_averages['50 SMA'] = data['Close'].rolling(window=50).mean()
            moving_averages['100 SMA'] = data['Close'].rolling(window=100).mean()
            moving_averages['200 SMA'] = data['Close'].rolling(window=200).mean()
        elif ma_type == 2:  # EMA
            moving_averages['8 EMA'] = data['Close'].ewm(span=8, adjust=False).mean()
            moving_averages['20 EMA'] = data['Close'].ewm(span=20, adjust=False).mean()
            moving_averages['50 EMA'] = data['Close'].ewm(span=50, adjust=False).mean()
            moving_averages['100 EMA'] = data['Close'].ewm(span=100, adjust=False).mean()
            moving_averages['200 EMA'] = data['Close'].ewm(span=200, adjust=False).mean()
        elif ma_type == 3:  # WMA
            moving_averages['8 WMA'] = data['Close'].rolling(window=8).apply(lambda x: np.average(x, weights=np.arange(1, 9)), raw=True)
            moving_averages['20 WMA'] = data['Close'].rolling(window=20).apply(lambda x: np.average(x, weights=np.arange(1, 21)), raw=True)
            moving_averages['50 WMA'] = data['Close'].rolling(window=50).apply(lambda x: np.average(x, weights=np.arange(1, 51)), raw=True)
            moving_averages['100 WMA'] = data['Close'].rolling(window=100).apply(lambda x: np.average(x, weights=np.arange(1, 101)), raw=True)
            moving_averages['200 WMA'] = data['Close'].rolling(window=200).apply(lambda x: np.average(x, weights=np.arange(1, 201)), raw=True)
        elif ma_type == 4:  # Hull
            def hull_moving_average(series, period):
                wma1 = series.rolling(window=period // 2).apply(lambda x: np.average(x, weights=np.arange(1, period // 2 + 1)), raw=True)
                wma2 = series.rolling(window=period).apply(lambda x: np.average(x, weights=np.arange(1, period + 1)), raw=True)
                return (2 * wma1 - wma2).rolling(window=int(np.sqrt(period))).mean()

            moving_averages['8 Hull'] = hull_moving_average(data['Close'], 8)
            moving_averages['20 Hull'] = hull_moving_average(data['Close'], 20)
            moving_averages['50 Hull'] = hull_moving_average(data['Close'], 50)
            moving_averages['100 Hull'] = hull_moving_average(data['Close'], 100)
            moving_averages['200 Hull'] = hull_moving_average(data['Close'], 200)

    return pd.DataFrame(moving_averages)

def score_moving_averages(data, moving_averages):
    scores = []
    for index, row in data.iterrows():
        score = 0
        for column in moving_averages.columns:
            if row['Close'] > moving_averages.at[index, column]:
                score += 1
        scores.append(score)
    data['Score'] = scores
    return data

# Ana fonksiyon
def main():
    # TradingView'den veri alma
    tv = TvDatafeed()
    Hisseler = get_all_symbols(market='turkey')
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]
    Hisseler = sorted(Hisseler)

    # Zaman dilimi sayısını seçme
    interval_count = get_interval_count_choice()

    # Kullanıcının seçimine göre zaman dilimlerini al
    selected_intervals = get_interval_choice(interval_count)

    # Kullanıcıdan hangi hareketli ortalamaları seçeceğini al
    ma_types = get_moving_average_choice()

    # Sonuçları depolamak için liste oluşturma
    results = []

    # Seçilen zaman dilimlerine göre analiz yapar
    for interval_data in selected_intervals:
        interval_name = interval_data[0]
        interval = interval_data[1]

        for symbol in Hisseler:
            try:
                # 500 barlık veri çekimi
                data = tv.get_hist(symbol=symbol, exchange='BIST', interval=interval, n_bars=500)
                data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
                data = data.reset_index()

                # Hareketli ortalamaları hesapla
                moving_averages = calculate_moving_averages(data, ma_types)

                # Puanlama yap
                scored_data = score_moving_averages(data, moving_averages)

                # Sonuçları topla
                last_row = scored_data.iloc[-1]
                new_row = {
                    'Kod': symbol,
                    'Kapanış': last_row['Close'],
                    'Puan': last_row['Score']
                }

                # Hareketli ortalamaları ekle
                for ma_column in moving_averages.columns:
                    new_row[ma_column] = moving_averages[ma_column].iloc[-1]

                results.append(new_row)

            except Exception as e:
                print(f"Error processing {symbol}: {e}")

    # Sonuçları DataFrame'e çevirme
    results_df = pd.DataFrame(results)

    # Sonuçları sıralama
    results_df = results_df.sort_values(by='Puan', ascending=False).reset_index(drop=True)

    # Ekranda sonuçları gösterme
    print("Hisse Senetleri Puanlama Sonuçları:")
    print(results_df)

    # Zaman dilimlerine göre sonuçları ayrı Excel dosyalarına kaydetme
    for idx, interval_data in enumerate(selected_intervals):
        interval_name = interval_data[0]
        excel_file = f'hisse_puanlama_sonuc_{interval_name}.xlsx'
        results_df.to_excel(excel_file, index=False)
        print(f"\nSonuçlar '{excel_file}' dosyasına kaydedildi.")

if __name__ == "__main__":
    main()

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-e9_5bp9z
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-e9_5bp9z
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done


Kaç farklı zaman diliminde tarama yapmak istiyorsunuz?
1: Tek Zaman Dilimi
2: İki Zaman Dilimi
Seçiminizi yapın (1-2): 1
Lütfen zaman dilimi(lerini) seçin:
1: 15 Dakika
2: 30 Dakika
3: 45 Dakika
4: 1 Saat
5: 2 Saat
6: 4 Saat
7: Günlük
8: Haftalık
9: Aylık
Birinci zaman dilimini seçin (1-9): 7
Hangi hareketli ortalamaları kullanmak istersiniz?
1: SMA
2: EMA
3: WMA
4: Hull
Seçimlerinizi yapın (örneğin: 1,2): 1
Hisse Senetleri Puanlama Sonuçları:
       Kod  Kapanış  Puan      8 SMA    20 SMA      50 SMA     100 SMA  \
0     DAGI     9.96     5    9.27375    9.1470    9.347000    9.053800   
1    TKFEN    65.65     5   59.10000   55.8895   54.329800   54.543200   
2    ALTIN    32.54     5   31.77625   31.4625   29.310800   26.977600   
3    KZBGY    31.64     5   31.12750   30.5730   27.290400   25.663800   
4     LOGO   104.60     5   99.83125   96.8175  101.952000  100.605000   
..     ...      ...   ...        ...       ...         ...         ...   
574  ORCAY     8.81     0    8.825